# EDA

In [224]:
data['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

is_there_an_emotion_directed_at_a_brand_or_product
No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: count, dtype: int64

In [225]:
# How do people feel about apple and google products

In [226]:
# How do they compare and which areas can apple improve in, compared to google?
# Product development 
# investment and a company choosing for best electronics to work with
# Brand image
# loyalty

In [227]:
# dopping null column from tweet_text column since it's only one and has no information for our analysis
data.tweet_text.isna().sum()
data.dropna(subset=['tweet_text'], inplace=True)

In [228]:
data.describe()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
count,9092,3291,9092
unique,9065,9,4
top,RT @mention Marissa Mayer: Google Will Connect...,iPad,No emotion toward brand or product
freq,5,946,5388


In [229]:
# looking at our .describe, there seems to be some duplicates in our data and we'll go ahead and keep unique inputs only
data.drop_duplicates(subset=['tweet_text'], inplace=True)

In [230]:
data.describe()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
count,9065,3280,9065
unique,9065,9,4
top,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPad,No emotion toward brand or product
freq,1,943,5372


In [231]:
# Creating a cleaned emotion_in tweet_is_directed_at column
data.rename(columns={'emotion_in_tweet_is_directed_at': 'brand'}, inplace=True)
data.rename(columns={'is_there_an_emotion_directed_at_a_brand_or_product': 'sentiment'}, inplace=True)

In [232]:
data['brand'].value_counts()

brand
iPad                               943
Apple                              659
iPad or iPhone App                 469
Google                             428
iPhone                             296
Other Google product or service    293
Android App                         80
Android                             77
Other Apple product or service      35
Name: count, dtype: int64

In [233]:

keywords = ["Apple", "Google", "iPad", "iPhone", "Android"]

for keyword in keywords:
    data[keyword] = data['tweet_text'].str.extract(f'({keyword})', flags=re.IGNORECASE)

In [234]:
data

,tweet_text,brand,sentiment,Apple,Google,iPad,iPhone,Android
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,NaN,NaN,NaN,iPhone,NaN
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,NaN,NaN,iPad,iPhone,NaN
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,NaN,NaN,iPad,NaN,NaN
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,NaN,NaN,NaN,iPhone,NaN
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,NaN,Google,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion,NaN,NaN,Ipad,NaN,NaN
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product,NaN,google,NaN,NaN,NaN
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product,NaN,Google,NaN,NaN,NaN
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product,NaN,NaN,NaN,iPhone,NaN


In [235]:
data.describe()

,tweet_text,brand,sentiment,Apple,Google,iPad,iPhone,Android
count,9065,3280,9065,2131,2462,2876,1569,574
unique,9065,9,4,3,3,6,6,3
top,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPad,No emotion toward brand or product,Apple,Google,iPad,iPhone,Android
freq,1,943,5372,1756,2035,2405,1338,439


In [236]:
# Next will combine apple products under apple and google products under
data['product'] = data[['Apple', 'iPhone', 'iPad', 'Google', 'Android']].apply(lambda x: 'Apple' if x[['Apple', 'iPhone', 'iPad']].any() 
                                                                                else 'Google' if x[['Google', 'Android']].any() 
                                                                                else None, axis=1)

In [237]:
data['product'].value_counts()

product
Apple     5517
Google    2764
Name: count, dtype: int64

In [238]:
# Since we've classified our product column, we'll drop nulls as we're interested with apple and google products
data['product'].isna().sum()

784

In [239]:
data.dropna(subset=['product'], inplace=True)

In [240]:
data

,tweet_text,brand,sentiment,Apple,Google,iPad,iPhone,Android,product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,NaN,NaN,NaN,iPhone,NaN,Apple
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,NaN,NaN,iPad,iPhone,NaN,Apple
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,NaN,NaN,iPad,NaN,NaN,Apple
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,NaN,NaN,NaN,iPhone,NaN,Apple
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,NaN,Google,NaN,NaN,NaN,Google
...,...,...,...,...,...,...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion,NaN,NaN,Ipad,NaN,NaN,Apple
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product,NaN,google,NaN,NaN,NaN,Google
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product,NaN,Google,NaN,NaN,NaN,Google
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product,NaN,NaN,NaN,iPhone,NaN,Apple


In [241]:
data['sentiment'].count()

8281

In [360]:
text = data.copy()

In [361]:
text['sentiment'].replace("I can't tell" , "nuetral emotion", inplace=True)

In [362]:
text['sentiment'].replace("No emotion toward brand or product" , "nuetral emotion", inplace=True)

In [363]:
text['sentiment'].value_counts(normalize=True)

sentiment
nuetral emotion     0.577587
Positive emotion    0.353822
Negative emotion    0.068591
Name: proportion, dtype: float64

In [364]:
text.groupby(['sentiment', 'product']).count()

tweet_text  brand  Apple  Google  iPad  iPhone  \
sentiment        product                                                   
Negative emotion Apple           419    389    116       1   189     161   
                 Google          149    129      0     134     0       0   
Positive emotion Apple          2125   1939    847      18  1198     546   
                 Google          805    698      0     668     0       0   
nuetral emotion  Apple          2973     72   1168      27  1489     862   
                 Google         1810     28      0    1614     0       0   

                          Android  
sentiment        product           
Negative emotion Apple          8  
                 Google        16  
Positive emotion Apple         54  
                 Google       142  
nuetral emotion  Apple        144  
                 Google       210

In [366]:
def preprocess_tweet(text):
    
    # Normalize Unicode characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Tokenize the text using regex
    tokens = re.findall(r'\b[A-Za-z]+\b', text.lower())
    
    # Remove stopwords
    stop_words = stopwords.words('english')
    stop_words.extend(["apple", "google", "ipad", "iphone", "android", "sxsw", "link", "mention"])
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join the tokens into a string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [370]:
print(data['tweet_text'][200])

rt ' It's 4 p.m. and the #iPad2 line at the Apple store is longer and wider ÛÒ about 250 people! Only one more hour. ' #sxsw


In [371]:
text['preprocess_tweets2'] = text['tweet_text'].apply(lambda x: preprocess_tweet(str(x)))
print(text['preprocess_tweets2'][200])

rt p line store longer wider uo people one hour


In [333]:
text['tokenized_tweet'] = text['preprocess_tweets2'].apply(lambda x: x.split() if isinstance(x, str) else [])
text[['preprocess_tweets2', 'tokenized_tweet']].head()

,preprocess_tweets2,tokenized_tweet
0,hr tweeting dead need upgrade plugin station,"[hr, tweeting, dead, need, upgrade, plugin, st..."
1,jessedee know fludapp awesome app likely appre...,"[jessedee, know, fludapp, awesome, app, likely..."
2,swonderlin wait also sale,"[swonderlin, wait, also, sale]"
3,hope year festival crashy year app,"[hope, year, festival, crashy, year, app]"
4,sxtxstate great stuff fri marissa mayer tim re...,"[sxtxstate, great, stuff, fri, marissa, mayer,..."


In [348]:
text['preprocess_tweets2'] = text['preprocess_tweets2'].apply(lambda x: x.encode())

In [349]:
type(text['preprocess_tweets2'])

pandas.core.series.Series

In [ ]:
nlp_object =  text['tokenized_tweet'] # Your NLP object (e.g., tokenized sentence or document)
preprocessed_text = preprocess_text(nlp_object)

In [253]:
text['tokenized_tweet'] = text['tweet_text'].apply(lambda x: x.split() if isinstance(x, str) else [])
text[['tweet_text', 'tokenized_tweet']].head()

,tweet_text,tokenized_tweet
0,i have a g iphone after hrs tweeting at rise...,"[i, have, a, g, iphone, after, hrs, tweeting, ..."
1,know about awesome ipadiphone app that youl...,"[know, about, awesome, ipadiphone, app, that, ..."
2,can not wait for ipad also they should sale ...,"[can, not, wait, for, ipad, also, they, should..."
3,i hope this years festival isnt as crashy as ...,"[i, hope, this, years, festival, isnt, as, cra..."
4,great stuff on fri sxsw marissa mayer google ...,"[great, stuff, on, fri, sxsw, marissa, mayer, ..."
